In [23]:
# CNN-RNN Model with Beam Search for Chest X-Ray Report Generation
# Modified to fix repetition issues and improve generation quality

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import matplotlib.pyplot as plt
from PIL import Image
import pickle
import time
import tqdm
import pydicom
import string
import certifi
import ssl
import heapq
import math

# Fix SSL certificate issues in macOS
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
os.environ['REQUESTS_CA_BUNDLE'] = certifi.where()

# Simple tokenizer to avoid NLTK dependency
def simple_tokenize(text):
    """Simple tokenizer that splits text on whitespace and punctuation."""
    if not isinstance(text, str):
        return []
    # Remove punctuation and replace with space
    for char in string.punctuation:
        text = text.replace(char, ' ' + char + ' ')

    # Split on whitespace and filter empty tokens
    tokens = [token for token in text.lower().split() if token.strip()]
    return tokens

# Define paths (keep these the same)
base_path = '/Users/simeon/Documents/DLH/content/mimic-cxr-project'
data_dir = os.path.join(base_path, 'data')
files_path = os.path.join(base_path, 'new_files')
output_dir = os.path.join(base_path, 'output')
reports_dir = os.path.join(base_path, 'reports')
models_dir = os.path.join(base_path, 'models')

# Create output directories
os.makedirs(output_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

# Import the report parser module
import sys
sys.path.append(f"{base_path}/modules")
from report_parser import parse_report, MIMIC_RE
print("Successfully imported report parser module")

# Load train and test data
train_df = pd.read_csv(os.path.join(data_dir, 'train.tsv'), sep='\t')
test_df = pd.read_csv(os.path.join(data_dir, 'test.tsv'), sep='\t')

print(f"Train data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Enhanced Vocabulary class with frequency logging
class Vocabulary:
    def __init__(self, freq_threshold=2):
        self.itos = {0: "<PAD>", 1: "< SOS >", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "< SOS >": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold
        self.frequencies = {}  # Track word frequencies

    def __len__(self):
        return len(self.itos)

    def build_vocabulary(self, sentence_list):
        self.frequencies = {}
        idx = 4

        for sentence in sentence_list:
            # Use simple tokenizer
            for word in simple_tokenize(sentence):
                if word not in self.frequencies:
                    self.frequencies[word] = 0
                self.frequencies[word] += 1

                if word not in self.stoi and self.frequencies[word] >= self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

        # Print top 20 most frequent words for debugging
        print("Top 20 most frequent words:")
        for word, freq in sorted(self.frequencies.items(), key=lambda x: x[1], reverse=True)[:20]:
            print(f"{word}: {freq}")

    def numericalize(self, text):
        # Use simple tokenizer
        tokenized_text = simple_tokenize(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]

# Class to load and preprocess the data
class ChestXRayReportDataset(Dataset):
    def __init__(self, df, is_train=True, transform=None, max_seq_length=100):
        self.df = df
        self.is_train = is_train
        self.transform = transform
        self.max_seq_length = max_seq_length
        self.reports = []

        # Extract reports if training
        if self.is_train:
            print("Extracting report texts for training data...")
            for _, row in tqdm.tqdm(self.df.iterrows(), total=len(self.df)):
                subject_id = row['subject_id']
                study_id = row['study_id']

                # Construct path to report (FIXED PATH)
                subject_prefix = f"p{str(subject_id)[:2]}"
                subject_dir = f"p{subject_id}"
                study_dir = f"s{study_id}"

                # Try different path structures
                report_paths = [
                    os.path.join(reports_dir, subject_prefix, subject_dir, f"{study_dir}.txt"),  # Original
                    os.path.join(reports_dir, 'new_files', subject_prefix, subject_dir, f"{study_dir}.txt"),  # With 'new_files'
                    os.path.join(reports_dir, 'files', subject_prefix, subject_dir, f"{study_dir}.txt"),  # With 'files'
                    os.path.join(reports_dir, subject_prefix, subject_dir, study_dir, "report.txt")  # Alternative structure
                ]

                for report_path in report_paths:
                    try:
                        if os.path.exists(report_path):
                            report = parse_report(report_path)
                            if 'findings' in report and report['findings']:
                                self.reports.append((row['dicom_id'], report['findings']))
                                break  # Found a valid report, move to next row
                    except Exception as e:
                        continue  # Try next path

            print(f"Extracted {len(self.reports)} reports from training data")

            # If still no reports, add some dummy data for testing
            if len(self.reports) == 0:
                print("WARNING: No reports found! Adding dummy data for testing...")
                # Create a few dummy reports to allow model testing
                for i in range(min(10, len(self.df))):
                    self.reports.append((self.df.iloc[i]['dicom_id'], "This is a dummy report for testing purposes."))

        # Build DICOM paths
        self.dicom_paths = []
        for _, row in self.df.iterrows():
            subject_id = row['subject_id']
            study_id = row['study_id']
            dicom_id = row['dicom_id']

            # Construct path to DICOM file
            subject_prefix = f"p{str(subject_id)[:2]}"
            subject_dir = f"p{subject_id}"
            study_dir = f"s{study_id}"
            dicom_file = f"{dicom_id}.dcm"
            dicom_path = os.path.join(files_path, subject_prefix, subject_dir, study_dir, dicom_file)

            if os.path.exists(dicom_path):
                self.dicom_paths.append((dicom_id, dicom_path))

    def __len__(self):
        return len(self.dicom_paths)

    def __getitem__(self, idx):
        dicom_id, dicom_path = self.dicom_paths[idx]

        # Load and transform image
        try:
            ds = pydicom.dcmread(dicom_path)
            pixel_array = ds.pixel_array

            # Normalize and convert to RGB
            pixel_array = pixel_array / np.max(pixel_array)
            img = np.uint8(pixel_array * 255)

            # Convert to RGB
            if len(img.shape) == 2:
                img_rgb = np.stack([img, img, img], axis=2)
            elif img.shape[2] == 1:
                img_rgb = np.concatenate([img, img, img], axis=2)
            else:
                img_rgb = img

            pil_img = Image.fromarray(img_rgb)
            if self.transform:
                image = self.transform(pil_img)
        except Exception as e:
            # Create blank image if loading fails
            image = torch.zeros(3, 224, 224)

        # Return image and report for training, only image for testing
        if self.is_train:
            report_text = ""
            for report_id, report in self.reports:
                if report_id == dicom_id:
                    report_text = report
                    break
            return image, report_text, dicom_id
        else:
            return image, dicom_id

    # Added method to get all reports for vocabulary building
    def get_all_reports(self):
        return [report for _, report in self.reports]

# Improved CNN-RNN model with attention and beam search
class CNNRNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, cnn_feature_size):
        super(CNNRNNModel, self).__init__()

        # CNN encoder (DenseNet121)
        print("Loading DenseNet121 model with pretrained weights...")
        self.densenet = models.densenet121(pretrained=True)
        self.densenet.classifier = nn.Linear(cnn_feature_size, embed_size)

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # Initial projection for CNN features
        self.feature_projection = nn.Linear(embed_size, hidden_size)

        # LSTM decoder
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)

        # Output layer
        self.fc = nn.Linear(hidden_size, vocab_size)

        # Dropout for regularization
        self.dropout = nn.Dropout(0.5)

        # Store dimensions
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.embed_size = embed_size

    def forward(self, images, captions=None, teacher_forcing_ratio=0.5):
        # Extract features from images using CNN encoder
        # [batch_size, 3, 224, 224] -> [batch_size, embed_size]
        features = self.densenet(images)

        # Project features to hidden state size
        projected_features = self.feature_projection(features)

        # Training mode with captions
        if captions is not None:
            batch_size = features.size(0)
            caption_length = captions.size(1)

            # Initialize tensor for storing outputs
            outputs = torch.zeros(batch_size, caption_length, self.vocab_size).to(device)

            # Initialize hidden and cell state with projected image features
            h = projected_features.unsqueeze(0)  # [1, batch_size, hidden_size]
            c = torch.zeros(1, batch_size, self.hidden_size).to(device)

            # First input to LSTM is the SOS token
            x = self.embedding(torch.ones(batch_size, 1, dtype=torch.long).to(device))  # [batch_size, 1, embed_size]

            # Generate caption one word at a time
            for t in range(caption_length):
                # Forward through LSTM
                lstm_out, (h, c) = self.lstm(x, (h, c))  # lstm_out: [batch_size, 1, hidden_size]

                # Apply simple attention (we'll skip complex attention for now to avoid dimension issues)
                # Forward through fully connected layer
                out = self.fc(lstm_out)  # [batch_size, 1, vocab_size]

                # Store output
                outputs[:, t] = out.squeeze(1)  # [batch_size, vocab_size]

                # Teacher forcing
                use_teacher_force = torch.rand(1).item() < teacher_forcing_ratio

                if use_teacher_force and t < caption_length - 1:
                    # Use ground truth as next input
                    x = self.embedding(captions[:, t+1].unsqueeze(1))  # [batch_size, 1, embed_size]
                else:
                    # Use predicted word as next input
                    pred_token = out.argmax(2)  # [batch_size, 1]
                    x = self.embedding(pred_token)  # [batch_size, 1, embed_size]

            return outputs

        # Inference mode (using beam search)
        else:
            return self.beam_search(images, beam_size=3, max_length=100)

    def beam_search(self, images, beam_size=3, max_length=100, temperature=1.0, repetition_penalty=1.2):
        """
        Beam search implementation for better text generation
        with repetition penalty to avoid word repetition
        """
        batch_size = images.size(0)
        results = []

        for idx in range(batch_size):
            # Extract features for a single image
            image_features = self.densenet(images[idx:idx+1])  # [1, embed_size]

            # Project features to hidden state size
            projected_features = self.feature_projection(image_features)  # [1, hidden_size]

            # Initialize hidden and cell states
            h = projected_features.unsqueeze(0)  # [1, 1, hidden_size]
            c = torch.zeros(1, 1, self.hidden_size).to(device)  # [1, 1, hidden_size]

            # Start with SOS token
            start_token = torch.ones(1, 1, dtype=torch.long).to(device)  # [1, 1]
            start_embedding = self.embedding(start_token)  # [1, 1, embed_size]

            # First forward pass to get initial predictions
            lstm_out, (h, c) = self.lstm(start_embedding, (h, c))  # lstm_out: [1, 1, hidden_size]

            # Get initial logits
            logits = self.fc(lstm_out)  # [1, 1, vocab_size]
            logits = logits.squeeze(0).squeeze(0)  # [vocab_size]

            # Apply temperature sampling
            logits = logits / temperature

            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # [vocab_size]

            # Get top-k candidates
            topk_probs, topk_indices = torch.topk(probs, beam_size)  # [beam_size]

            # Initialize beams
            beams = [(math.log(prob.item()), [index.item()], h, c, {index.item(): 1})
                     for prob, index in zip(topk_probs, topk_indices)]

            # Beam search
            for _ in range(max_length - 1):
                new_beams = []

                # Expand each beam
                for log_prob, seq, hidden, cell, word_counts in beams:
                    # Check if sequence ended
                    if seq[-1] == 2:  # <EOS> token
                        new_beams.append((log_prob, seq, hidden, cell, word_counts))
                        continue

                    # Prepare input for next step (last token)
                    token = torch.tensor([[seq[-1]]], dtype=torch.long).to(device)
                    token_embedding = self.embedding(token)  # [1, 1, embed_size]

                    # Forward pass
                    lstm_out, (new_hidden, new_cell) = self.lstm(token_embedding, (hidden, cell))

                    # No attention for simplicity

                    # Get logits
                    logits = self.fc(lstm_out).squeeze(0).squeeze(0)  # [vocab_size]

                    # Apply repetition penalty
                    for word_idx, count in word_counts.items():
                        logits[word_idx] /= repetition_penalty * count

                    # Apply temperature
                    logits = logits / temperature

                    # Apply softmax
                    probs = F.softmax(logits, dim=-1)  # [vocab_size]

                    # Get top-k candidates for this beam
                    topk_probs, topk_indices = torch.topk(probs, beam_size)  # [beam_size]

                    # Add to new beams
                    for prob, index in zip(topk_probs, topk_indices):
                        # Create new sequence
                        new_seq = seq + [index.item()]

                        # Update word counts for repetition penalty
                        new_word_counts = word_counts.copy()
                        if index.item() in new_word_counts:
                            new_word_counts[index.item()] += 1
                        else:
                            new_word_counts[index.item()] = 1

                        # Calculate new log probability
                        new_log_prob = log_prob + math.log(prob.item())

                        # Add beam
                        new_beams.append((new_log_prob, new_seq, new_hidden, new_cell, new_word_counts))

                # Keep top beams
                beams = sorted(new_beams, key=lambda x: x[0], reverse=True)[:beam_size]

                # Check if all beams end with <EOS>
                if all(seq[-1] == 2 for _, seq, _, _, _ in beams):
                    break

            # Get best beam
            best_beam = max(beams, key=lambda x: x[0])
            best_seq = best_beam[1]

            # Remove < SOS > and <EOS> tokens if present
            if best_seq[0] == 1:  # < SOS >
                best_seq = best_seq[1:]
            if best_seq[-1] == 2:  # <EOS>
                best_seq = best_seq[:-1]

            # Add to results
            results.append(torch.tensor(best_seq))

        # Create padded tensor
        max_len = max([len(seq) for seq in results]) if results else 0
        padded_results = torch.zeros(batch_size, max_len, dtype=torch.long)
        for i, seq in enumerate(results):
            end = len(seq)
            padded_results[i, :end] = seq

        return padded_results

# Function to generate dummy data for testing the model
def create_dummy_batch(batch_size=2, seq_len=10, vocab_size=100):
    # Create dummy images
    images = torch.randn(batch_size, 3, 224, 224)

    # Create dummy captions
    captions = torch.randint(0, vocab_size, (batch_size, seq_len))

    return images, captions

# Main execution function
def run(training_mode=False):
    # Create dataset for training
    print("Creating training dataset...")
    train_dataset = ChestXRayReportDataset(train_df, is_train=True, transform=transform)

    # Build vocabulary - FIXED: Use the get_all_reports method
    print("Building vocabulary...")
    vocab = Vocabulary()
    # Get all reports from the dataset
    all_reports = train_dataset.get_all_reports()
    vocab.build_vocabulary(all_reports)
    print(f"Built vocabulary with {len(vocab)} tokens")

    # Save vocabulary
    vocab_path = os.path.join(models_dir, 'vocab.pkl')
    with open(vocab_path, 'wb') as f:
        pickle.dump(vocab, f)

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=32,
        shuffle=True,
        num_workers=0  # Set to 0 to avoid multiprocessing issues on Mac
    )

    # Initialize model
    model = CNNRNNModel(
        vocab_size=len(vocab),
        embed_size=256,
        hidden_size=512,
        cnn_feature_size=1024
    )

    # Move model to device
    model = model.to(device)

    # Test model with dummy data to verify dimensions
    print("Testing model with dummy data...")
    dummy_images, dummy_captions = create_dummy_batch(batch_size=2, seq_len=10, vocab_size=len(vocab))
    dummy_images = dummy_images.to(device)
    dummy_captions = dummy_captions.to(device)

    # Forward pass
    try:
        with torch.no_grad():
            outputs = model(dummy_images, dummy_captions)
            print(f"Dummy forward pass successful! Output shape: {outputs.shape}")
    except Exception as e:
        print(f"Error in dummy forward pass: {e}")
        return

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding tokens
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=16, gamma=0.5)  # LR decay every 16 epochs

    # Load model path (but we won't load the model as requested)
    model_path = os.path.join(models_dir, 'cnn_rnn_improved.pth')

    if training_mode:
        print("Training new model from scratch...")
        # Training loop
        num_epochs = 64  # Change to 64 for full training
        best_loss = float('inf')

        for epoch in range(num_epochs):
            model.train()
            train_loss = 0

            # Gradually decrease teacher forcing
            teacher_forcing_ratio = max(0.5 - (epoch // 16) * 0.1, 0.0)

            for i, (images, captions, _) in enumerate(tqdm.tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
                images = images.to(device)

                # Tokenize and pad captions
                tokenized_captions = []
                for caption in captions:
                    if caption:
                        tokens = [1]  # < SOS >
                        tokens.extend(vocab.numericalize(caption))
                        tokens.append(2)  # <EOS>
                    else:
                        tokens = [1, 2]  # < SOS >, <EOS>
                    tokenized_captions.append(tokens)

                # Pad sequences
                padded_captions = []
                for tokens in tokenized_captions:
                    if len(tokens) > 100:
                        padded_captions.append(tokens[:100])
                    else:
                        padded_captions.append(tokens + [0] * (100 - len(tokens)))

                captions_tensor = torch.tensor(padded_captions).to(device)

                # Zero the gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(images, captions_tensor, teacher_forcing_ratio)

                # Reshape for loss calculation
                outputs = outputs.reshape(-1, outputs.shape[2])
                targets = captions_tensor.reshape(-1)

                # Calculate loss
                loss = criterion(outputs, targets)

                # Backward pass
                loss.backward()

                # Clip gradients
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

                # Update weights
                optimizer.step()

                # Update training loss
                train_loss += loss.item()

                # Print progress
                if (i + 1) % 50 == 0:
                    print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

                # Break after a few batches for testing
                #if i >= 3:  # Remove this line for full training
                #    break

            # Update learning rate
            scheduler.step()

            # Calculate average loss for the epoch
            avg_loss = train_loss / len(train_loader)  # Calculate average over all batches
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Teacher forcing: {teacher_forcing_ratio:.2f}")

            # Save model if it's the best so far
            if avg_loss < best_loss:
                best_loss = avg_loss
                torch.save(model.state_dict(), model_path)
                print(f"Model saved with improved loss: {best_loss:.4f}")

    # Create test dataset
    print("Creating test dataset...")
    test_dataset = ChestXRayReportDataset(test_df, is_train=False, transform=transform)

    test_loader = DataLoader(
        test_dataset,
        batch_size=1,  # Process one at a time for generation
        shuffle=False,
        num_workers=0  # Set to 0 to avoid multiprocessing issues on Mac
    )

    # Generate reports using beam search
    print("Generating reports using beam search...")
    model.eval()

    generated_reports = {}

    with torch.no_grad():
        # Process a few samples for demonstration
        max_samples = 3
        sample_count = 0

        for images, dicom_ids in tqdm.tqdm(test_loader):
            if sample_count >= max_samples:
                break

            images = images.to(device)
            dicom_id = dicom_ids[0]  # Batch size is 1

            try:
                # Generate caption using beam search
                outputs = model(images)  # Uses beam search by default

                # Convert token indices to words
                caption = []
                for token_id in outputs[0]:
                    token_id = token_id.item()
                    if token_id == 2:  # <EOS>
                        break
                    if token_id > 3:  # Skip <PAD>, < SOS >, <EOS>, <UNK>
                        caption.append(vocab.itos[token_id])

                # Join words into a sentence, properly handling punctuation
                report_text = ""
                for word in caption:
                    if word in string.punctuation:
                        report_text += word
                    else:
                        if report_text and not report_text.endswith(' '):
                            report_text += ' '
                        report_text += word

                generated_reports[dicom_id] = report_text

                sample_count += 1
                print(f"Generated report {sample_count}/{max_samples}")
            except Exception as e:
                print(f"Error generating report for {dicom_id}: {e}")

    # Show sample reports
    print("\nSample generated reports:")
    for dicom_id, report in generated_reports.items():
        print(f"\nDICOM ID: {dicom_id}")
        print(f"Report: {report}")

    # Save generated reports
    if generated_reports:
        report_df = pd.DataFrame({
            'dicom_id': list(generated_reports.keys()),
            'generated': list(generated_reports.values())
        })

        output_file = os.path.join(output_dir, 'cnn_rnn_beam_search.tsv')
        report_df.to_csv(output_file, sep='\t', index=False)

        print(f"Generated {len(generated_reports)} reports and saved to {output_file}")
    else:
        print("No reports were generated")

# For this example, setting training_mode to True to train a new model
# Change to False to only generate reports using a pretrained model
if __name__ == "__main__":
    run(training_mode=True)

Successfully imported report parser module
Train data shape: (4370, 3)
Test data shape: (1688, 3)
Using device: cpu
Creating training dataset...
Extracting report texts for training data...


100%|██████████| 4370/4370 [00:01<00:00, 2950.77it/s]


Extracted 4362 reports from training data
Building vocabulary...
Top 20 most frequent words:
.: 24934
the: 15302
is: 12317
,: 5867
no: 4919
are: 4840
there: 4762
of: 4663
right: 4461
and: 4139
in: 3957
left: 3548
pleural: 3147
pneumothorax: 3025
effusion: 2869
lung: 2539
to: 2499
a: 2433
with: 2416
or: 2239
Built vocabulary with 1925 tokens
Loading DenseNet121 model with pretrained weights...
Testing model with dummy data...
Dummy forward pass successful! Output shape: torch.Size([2, 10, 1925])
Training new model from scratch...


Epoch 1/64:  36%|███▋      | 50/137 [08:17<14:34, 10.05s/it]

Epoch [1/64], Step [50/137], Loss: 4.2419


...

Epoch [64/64], Loss: 3.2178, Teacher forcing: 0.20
Creating test dataset...
Generating reports using beam search...


  0%|          | 2/1688 [00:00<03:53,  7.22it/s]

Generated report 1/3
Generated report 2/3


  0%|          | 3/1688 [00:00<04:28,  6.27it/s]


Generated report 3/3

Sample generated reports:

DICOM ID: b8e14e3b-545cd663-a00812c0-9e772d64-b3d40e32
Report: portable portable upright view of the chest. there ekg leads are present. the lungs lungs appear appear clear without without focal or or pneumothorax effusion effusion

DICOM ID: 8c295118-0d590369-21de2213-312374d8-50c8a349
Report: portable portable upright view of the chest. there ekg leads are present. the lungs lungs appear appear clear without without focal or or pneumothorax effusion effusion

DICOM ID: fc75e61c-ee134385-a5d0e01b-695f8125-2ed13ad2
Report: portable portable upright view of the chest. there ekg leads are present. the lungs lungs appear appear clear without without focal or or pneumothorax effusion effusion
Generated 3 reports and saved to /Users/simeon/Documents/DLH/content/mimic-cxr-project/output/cnn_rnn_beam_search.tsv
